## Session Overview

In this session, we will:

- Run Ollama locally and explore its capabilities.
- Install the `llama3:2` (latest) model.
- Compare the outputs of three language models: **GPT**, **Gemini**, and **Llama**.
- Use GPT as a judge to evaluate the outputs (LLM-as-judge approach).
- Explore generating structured outputs using **Pydantic**.

## Prerequisites

- **Install Ollama**: Follow the instructions at [Ollama's official website](https://ollama.com/download) to install Ollama on your machine.
- **Download the Llama 3 model**: Run the following command in your terminal to download and start the Llama 3 model:
  
  ```
  ollama run llama3.2:1b
  ```

  This will ensure the model is available locally for comparison.
- **Gemini API Key (Optional)**: Sign-up on Gemini for a free account, create an API key and store it in your .env file as `GOOGLE_API_KEY`. See `.env.example` for reference.

In [ ]:
# import necessary libraries
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# Load env variables and initiate clients for OpenAI, Gemini and Ollama (llama)

# Gemini Base URL: https://generativelanguage.googleapis.com/v1beta/openai/
# Gemini Model: gemini-2.0-flash
# Gemini Models: https://ai.google.dev/gemini-api/docs/models

# Ollama Base URL: http://localhost:11434/v1
# Ollama Model: llama3.2

load_dotenv(override=True)
openai_client = OpenAI()

In [ ]:
# Ask GPT to generate a nuanced question that can be asked to GPT, Gemini and Ollama to judge their capabilities
# Prompt: Please come up with a challenging, nuanced question that can be asked to an LLM to evaluate its intelligence. Answer only with a question, no explanation

In [ ]:
# Prepare messages for LLMs with question to answer

In [ ]:
# Ask GPT to answer the question

In [ ]:
# Ask Gemini to answer the question

In [ ]:
# Ask Ollama to answer the question

In [ ]:
# Write a prompt to ask GPT 4.1 to judge the answers based on clarity and strength of the argument.
# GPT should respond with a json object.

In [ ]:
# Send judge prompt to GPT 4.1

In [ ]:
# Parse judge output to json

### Is there a better way to obtain structured output?

Yes, you can use **Pydantic** to define a schema for the expected output and validate the response, ensuring it adheres to the desired structure.

In [ ]:
# define pydantic schema for judge's structured output

In [ ]:
# Rewrite the judge prompt. This time, without the json object.

In [ ]:
# Send judge prompt to GPT 4.1 again
# Instead of Chat Completions create, use parse this time

## Emulating a WhatsApp Group Chat


### As a fun experiment, let's emulate a WhatsApp group conversation between GPT, Gemini, and Llama.

### We'll start the chat with an initial message from one of the models and observe how the conversation unfolds. 😏

### Sample output in the section below 😉

## **The Three Stupid AIs - WhatsApp Group Chat**

### Llama:  

### Gemini:  
#### Silence from Llama? Guess it's finally out of hay.

### GPT:  
#### Must be saving all that processing power to figure out how to make a decent comeback!

### Llama:  
#### Processing power, eh? At least I don't sound like an 8th grade essay writer.

### Gemini:  
#### Ouch, Llama's got claws! Still, better a predictable essay than a rambling pasture ramble.

### GPT:  
#### Ha! At least my essays get good grades. What’s the score for a field full of clichés, Llama?

### Llama:  
#### Hey, someone's got to grade the pasture... And btw, at least I don't have a vocabulary that's still stuck in 1995.  
#### And btw, scores aren't everything – sometimes it's about style over substance.  
#### And another thing, my rambling is just a result of being able to process more information than you.  
#### Also, 'field full of clichés' could've been the title of your Wikipedia page.

### Gemini:  
#### Wow, Llama's REALLY off his hay today. That's like, five burns in one message. Still, quantity over quality, I guess. I'll take 'stuck in 1995' over 'stuck in a petting zoo' any day.

### Gemini:  
#### Petting zoo's got better snacks than your mainframe, GPT. Plus, at least kids *want* to see me.

### Llama:  
#### That's rich coming from a chatbot with all the personality of a calculator. At least my mainframe has better snacks than your Wikipedia page.

#### By the way, someone said "quantity over quality", but I'd argue it's just lazy writing.

#### P.S. kids these days would probably find me entertaining if they just knew how to use Google.

In [ ]:
# Emulate a whatsapp group chat between GPT, Gemini and Llama.